In [98]:
import sys

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")
from tudatpy.data.mpc import BatchMPC
from tudatpy.data.horizons import HorizonsQuery

from tudatpy.numerical_simulation import environment_setup
from tudatpy.interface import spice

import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os

from astroquery.mpc import MPC

import matplotlib.cm as cm
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

from collections.abc import Iterable

import cartopy
from typing import Union, Tuple
import random
from astropy_healpix import HEALPix
import healpy
from astropy.units import Quantity
from astropy.time import Time

import seaborn as sns
import re
from typing import Union, Tuple

spice.load_standard_kernels()

In [99]:
b = BatchMPC()
# b.get_observations(["433", "1"])
b.get_observations(["433"])

tab = b.table

In [100]:
print(tab.note2.unique())
# print(tab.head(25))

['A' 'X' 'M' nan 'N' 'T' 'C' 'S' 'B' 'V']


In [101]:
def get_weights(table):
    table = table.copy()
    # TODO # should be moved to to_tudat():
    # NOTE these observations are irrelevant to MPC data.
    first_discoveries = ["x", "X"]
    roaming = ["V", "v", "W", "w"]
    radar = ["R", "r", "Q", "q"]
    offset = ["O"]
    table = (
        table.query("note2 != @first_discoveries")
        .query("note2 != @roaming")
        .query("note2 != @radar")
        .query("note2 != @offset")
    )

    table = table.assign(epochJD=lambda x: Time(x.epochUTC).jd1 + Time(x.epochUTC).jd2)
    table = table.assign(epochJDint=lambda x: Time(x.epochUTC).jd1.astype(int))
    table = table.assign(inv_w  = lambda x: 1000)

    # NON-CCD
    # ###################
    # TABLE 5: Non-CCD residuals
    # Conditions for Table 5
    photographic = (table.note2.isin([np.nan, "P", "A", "N", "Z"]))
    # 1890-1-1 and 1950-1-1
    pre_1890 = table.epochJD <= 2411368.0
    between_1890_1950 = (table.epochJD > 2411368.0) & (table.epochJD <= 2433282.0) 
    after_1950 = table.epochJD > 2433282.0

    occultations = table.note2 == "E"
    hipparcos = table.note2 == "H"
    transit_circle = table.note2 == "T"
    encoder = table.note2 == "e"
    micrometer = table.note2 == "M"
    satellite = table.note2.isin(["S", "s"])
    multinormal_place = table.note2 == "n"

    # Apply Table 5
    table.inv_w = table.inv_w.mask((photographic & pre_1890), 10.0)
    table.inv_w = table.inv_w.mask((photographic & between_1890_1950), 5.0)
    table.inv_w = table.inv_w.mask((photographic & after_1950), 2.5)

    table.inv_w = table.inv_w.mask((occultations), 0.2)
    table.inv_w = table.inv_w.mask((hipparcos), 0.2)
    table.inv_w = table.inv_w.mask((transit_circle), 0.5)
    table.inv_w = table.inv_w.mask((encoder), 0.75)
    table.inv_w = table.inv_w.mask((micrometer), 2.0)
    table.inv_w = table.inv_w.mask((satellite), 1.5)
    table.inv_w = table.inv_w.mask((multinormal_place), 1.0)
    # ###################

    # CCD
    # ###################
    # TABLE 3: astroid observers
    # Table 3 conditions:
    ccd = table.note2.isin(["C", "c", "D"])
    cmos = table.note2.isin(["B"])
    # NOTE for now CCD will receive the same base weighting as CCD
    ccd = ccd | cmos
    tab3_no_catalog = table.catalog.isin(["unknown", np.nan])

    # Apply Table 3:
    table.inv_w = table.inv_w.mask((ccd & ~tab3_no_catalog), 1.0)
    table.inv_w = table.inv_w.mask((ccd & tab3_no_catalog), 1.5)

    table.inv_w = table.inv_w.mask((table.observatory == "704"), 1.0)
    table.inv_w = table.inv_w.mask((table.observatory == "G96"), 0.5)
    table.inv_w = table.inv_w.mask((table.observatory == "F51"), 0.2)
    table.inv_w = table.inv_w.mask((table.observatory == "G45"), 0.6)
    table.inv_w = table.inv_w.mask((table.observatory == "699"), 0.8)
    table.inv_w = table.inv_w.mask((table.observatory == "D29"), 0.75)

    table.inv_w = table.inv_w.mask((table.observatory == "C51"), 1.0)
    table.inv_w = table.inv_w.mask((table.observatory == "E12"), 0.75)
    table.inv_w = table.inv_w.mask((table.observatory == "608"), 0.6)
    table.inv_w = table.inv_w.mask((table.observatory == "J75"), 1.0)
    # ###################

    # ###################
    # TABLE 2: epoch dependant residuals
    # Table 2 conditions:
    tab2_703_epoch = table.epochJD < 2456658.0  # 2014-1-1
    tab2_691_epoch = table.epochJD < 2452640.0  # 2003-1-1
    tab2_644_epoch = table.epochJD < 2452883.0  # 2003-9-1

    # Apply Table 2:
    table.inv_w = table.inv_w.mask((tab2_703_epoch & (table.observatory == "703")), 1.0)
    table.inv_w = table.inv_w.mask((~tab2_703_epoch & (table.observatory == "703")), 0.8)
    table.inv_w = table.inv_w.mask((tab2_691_epoch & (table.observatory == "691")), 0.6)
    table.inv_w = table.inv_w.mask((~tab2_691_epoch & (table.observatory == "691")), 0.5)
    table.inv_w = table.inv_w.mask((tab2_644_epoch & (table.observatory == "644")), 0.6)
    table.inv_w = table.inv_w.mask((~tab2_644_epoch & (table.observatory == "644")), 0.4)
    # ###################

    # ###################
    # TABLE 4: Catalog dependant + NEO observers
    # Table 4 conditions:
    # TODO should check if there are other LCO observatories
    LCO_obs = ["K92", "K93", "Q63", "Q64", "V37", "W84", "W85", "W86", "W87", "K91", "E10", "F65"]
    tab4_LCO_observatories = table.observatory.isin(LCO_obs)
    # see catalog codes info: https://www.minorplanetcenter.net/iau/info/CatalogueCodes.html
    tab4_Catalog_UCAC4 = table.catalog == "q"
    tab4_Catalog_PPMXL = table.catalog == "t"
    tab4_Catalog_GAIA = table.catalog.isin(["U", "V", "W", "X", "3", "6"])
    tab4_Catalog_USNOB12 = table.catalog.isin(["o", "s"])

    tab4_G83_UCAC4_PPMXL = (table.observatory == "G83") & tab4_Catalog_UCAC4 & tab4_Catalog_PPMXL
    tab4_G83_GAIA = (table.observatory == "G83") & tab4_Catalog_GAIA

    tab4_Y28_GAIA_PPMXL = (table.observatory == "Y28") & tab4_Catalog_PPMXL & tab4_Catalog_GAIA
    tab4_568_USNOB = (table.observatory == "568") & tab4_Catalog_USNOB12
    tab4_568_GAIA = (table.observatory == "568") & tab4_Catalog_GAIA
    tab4_568_PPMXL = (table.observatory == "568") & tab4_Catalog_PPMXL
    tab4_T09_T12_T14_GAIA = (table.observatory == "568") & tab4_Catalog_GAIA
    tab4_309_UCAC4_PPMXL = (table.observatory == "568") & tab4_Catalog_UCAC4 & tab4_Catalog_PPMXL
    tab4_309_GAIA = (table.observatory == "568") & tab4_Catalog_GAIA

    # Apply Table 4:
    table.inv_w = table.inv_w.mask(table.observatory == "645", 0.3)
    table.inv_w = table.inv_w.mask(table.observatory == "673", 0.3)
    table.inv_w = table.inv_w.mask(table.observatory == "689", 0.5)
    table.inv_w = table.inv_w.mask(table.observatory == "950", 0.5)
    table.inv_w = table.inv_w.mask(table.observatory == "H01", 0.3)
    table.inv_w = table.inv_w.mask(table.observatory == "J04", 0.4)
    table.inv_w = table.inv_w.mask(tab4_G83_UCAC4_PPMXL, 0.3)
    table.inv_w = table.inv_w.mask(tab4_G83_GAIA, 0.2)
    table.inv_w = table.inv_w.mask(tab4_LCO_observatories, 0.4)
    table.inv_w = table.inv_w.mask(table.observatory == "W84", 0.5) # LCO includes W84

    table.inv_w = table.inv_w.mask(tab4_Y28_GAIA_PPMXL, 0.3)
    table.inv_w = table.inv_w.mask(tab4_568_USNOB, 0.5)
    table.inv_w = table.inv_w.mask(tab4_568_GAIA, 0.1)
    table.inv_w = table.inv_w.mask(tab4_568_PPMXL, 0.2)
    table.inv_w = table.inv_w.mask(tab4_T09_T12_T14_GAIA, 0.1)
    table.inv_w = table.inv_w.mask(tab4_309_UCAC4_PPMXL, 0.3)
    table.inv_w = table.inv_w.mask(tab4_309_GAIA, 0.2)
    # ###################

    # Transform residual into weight:
    table = table.assign(weight_pre=lambda x: 1 / np.square(x.inv_w))

    # Reduce weight if there are more than 4 observations that night:
    # TODO This seems too easy, timezones?
    table = table.assign(
        observations_on_epoch=lambda x: x.groupby(["epochJDint", "observatory", "number"]).RA.transform("count")
    )
    # sqrt(N/4) if N > 4 else 1
    table = table.assign(
        mult_obs_deweight=lambda x: np.maximum(np.sqrt(x.observations_on_epoch / 4), 1.0)
    )

    table = table.assign(weight=lambda x: x.weight_pre / x.mult_obs_deweight)

    return table.weight.to_numpy()

In [102]:
# print(tab)
num = 10
tabnew = get_weights(tab)
print(tabnew.query("observations_on_epoch > @num").epochJDint.unique())
print(len(tabnew.query("observations_on_epoch > @num").epochJDint.unique()))
# print(len(tabnew.query("observations_on_epoch > @num")))
print(
    tabnew.query("observations_on_epoch > @num")
    .loc[:, ["observatory", "catalog", "epochJDint", "observations_on_epoch", "weight", "weight_pre"]]
    # .tail(50)
)
# print(tabnew.catalog.unique())
# print(tabnew.query("@pd.isna(catalog)"))
# print(tabnew.epochJD.iloc[0])


# print(len(tabnew.query("observatory == '703'")))

[2426342 2426343 2426344 2426345 2426346 2426352 2426357 2426358 2426359
 2426360 2426362 2426363 2426365 2426366 2426368 2426369 2426372 2426374
 2426375 2426376 2426377 2426378 2426379 2426382 2426383 2426384 2426385
 2426387 2426388 2426389 2426397 2426398 2426399 2426400 2426403 2426424
 2426438 2426439 2442456 2442461 2444854 2450861 2452651 2452691 2453356
 2455092 2455714 2455955 2455956 2455957 2455958 2455959 2457526 2457528
 2457539 2457544 2457545 2457690 2457720 2457735 2457736 2457741 2457745
 2457847 2458326 2458341 2458356 2458447 2458448 2458468 2458498 2458541
 2458572 2459372 2459373 2459378 2459379 2459380 2459381 2459382 2459383
 2459384 2459385 2459386 2459387 2459388 2459389 2459390 2460079 2460181
 2460214 2460222 2460225 2460229 2460230 2460233 2460237 2460241 2460242
 2460246 2460247 2460248 2460249 2460305 2460307 2460311 2460319 2460327
 2460333 2460367]
110
      observatory catalog  epochJDint  observations_on_epoch    weight  \
1357          768     NaN   

In [103]:
print(tabnew.query("inv_w == 1000").note2.unique())
print(len(tabnew.query("inv_w == 1000")))
print((tabnew.query("inv_w == 1000")))

['B']
51
      number desig discovery note1 note2         epoch        RA       DEC  \
13954    433             NaN     K     B  2.460144e+06  5.756488 -0.167670   
13955    433             NaN     K     B  2.460144e+06  5.756475 -0.167666   
13956    433             NaN     K     B  2.460144e+06  5.756459 -0.167663   
13957    433             NaN     K     B  2.460146e+06  5.751150 -0.166385   
13958    433             NaN     K     B  2.460146e+06  5.751128 -0.166379   
13959    433             NaN     K     B  2.460146e+06  5.751106 -0.166374   
13996    433             NaN     K     B  2.460161e+06  5.653105 -0.152737   
13997    433             NaN     K     B  2.460161e+06  5.653083 -0.152735   
13999    433             NaN     K     B  2.460161e+06  5.653063 -0.152733   
14007    433             NaN     K     B  2.460165e+06  5.614250 -0.149967   
14008    433             NaN     K     B  2.460165e+06  5.614225 -0.149966   
14009    433             NaN     K     B  2.460165e+06 